* **Custom module**

In [13]:
import os
from os.path import exists
import pickle
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import backend as K
from sklearn.model_selection import cross_val_score, KFold

import pickle
import gc
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt

* **Paper module**

In [ ]:
import pickle
import tensorflow as tf
from keras import models
from keras.layers import Dense, Merge
from keras.callbacks import EarlyStopping
from keras import backend as K
import numpy as np
import time
import os

In [2]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction/prediction/src
/home/wmbio/WORK/gitworking/Dependency_prediction


In [14]:
def load_data(filename):
    data = []
    gene_names = []
#     data_labels = []
    lines = open(filename).readlines()
    sample_names = lines[0].replace('\n', '').split('\t')[1:]
    dx = 1

    for line in tqdm(lines[dx:], leave=True):
        values = line.replace('\n', '').split('\t')
        gene = str.upper(values[0])
        gene_names.append(gene)
        data.append(values[1:])
    data = np.array(data, dtype='float32')
    data = np.transpose(data)

    return data, sample_names, gene_names

def exp_model():
    with tf.device('/cpu:0'):
        model_exp = Sequential()
        model_exp.add(Dense(500, input_dim=premodel_exp[0][0].shape[0], activation=activation_func,
                            weights=premodel_exp[0], trainable=True))
        model_exp.add(Dense(200, input_dim=500, activation=activation_func, weights=premodel_exp[1],
                            trainable=True))
        model_exp.add(Dense(50, input_dim=200, activation=activation_func, weights=premodel_exp[2],
                            trainable=True))

        # subnetwork of gene fingerprints
        model_gene = Sequential()
        model_gene.add(Dense(1000, input_dim=data_fprint.shape[1], activation=activation_func, kernel_initializer=init,
                             trainable=True))
        model_gene.add(Dense(100, input_dim=1000, activation=activation_func, kernel_initializer=init, trainable=True))
        model_gene.add(Dense(50, input_dim=100, activation=activation_func, kernel_initializer=init, trainable=True))

        conc = Concatenate()([model_exp.output, model_gene.output])

        model_pre = Dense(dense_layer_dim, input_dim=100, activation=activation_func, kernel_initializer=init,
                              trainable=True)(conc)
        model_pre = Dense(dense_layer_dim, input_dim=dense_layer_dim, activation=activation_func, kernel_initializer=init,
                              trainable=True)(model_pre)
        model_pre = Dense(1, input_dim=dense_layer_dim, activation=activation_func2, kernel_initializer=init,
                              trainable=True)(model_pre)

        model_final = Model([model_exp.input, model_gene.input], model_pre)
        
        return model_final  

def exp_model_paper():
    with tf.device('/cpu:0'):
        model_exp = models.Sequential()
        model_exp.add(Dense(output_dim=500, input_dim=premodel_exp[0][0].shape[0], activation=activation_func,
                            weights=premodel_exp[0], trainable=True))
        model_exp.add(Dense(output_dim=200, input_dim=500, activation=activation_func, weights=premodel_exp[1],
                            trainable=True))
        model_exp.add(Dense(output_dim=50, input_dim=200, activation=activation_func, weights=premodel_exp[2],
                            trainable=True))

        model_gene = models.Sequential()
        model_gene.add(Dense(output_dim=1000, input_dim=data_fprint.shape[1], activation=activation_func, init=init,
                             trainable=True))
        model_gene.add(Dense(output_dim=100, input_dim=1000, activation=activation_func, init=init, trainable=True))
        model_gene.add(Dense(output_dim=50, input_dim=100, activation=activation_func, init=init, trainable=True))

        model_final = models.Sequential()
        model_final.add(Merge([model_exp, model_gene], mode='concat'))
        model_final.add(Dense(output_dim=dense_layer_dim, input_dim=100, activation=activation_func, init=init))
        model_final.add(Dense(output_dim=dense_layer_dim, input_dim=dense_layer_dim, activation=activation_func, init=init))
        model_final.add(Dense(output_dim=1, input_dim=dense_layer_dim, activation=activation_func2, init=init))
        
        return model_final  
    

class generator(tf.keras.utils.Sequence):
    def __init__(self, x_mut, x_exp, x_cna, x_meth, x_fprint, y_dep, batch_size):
        self.x_mut, self.x_exp, self.x_cna, self.x_meth, self.x_fprint, self.y_dep = x_mut, x_exp, x_cna, x_meth, x_fprint, y_dep
        self.bs = batch_size
    
    def __len__(self):
        return (len(self.y_dep) - 1) // self.bs + 1
    
    def __getitem__(self,idx):
        start, end = idx * self.bs, (idx+1) * self.bs
        return (self.x_mut[start:end], self.x_exp[start:end], self.x_cna[start:end], self.x_meth[start:end], self.x_fprint[start:end]), self.y_dep[start:end]

class generator3(generator):
    def __init__(self, x_mut, x_exp, x_cna, x_fprint, y_dep, batch_size):
        self.x_mut, self.x_exp, self.x_cna, self.x_fprint, self.y_dep = x_mut, x_exp, x_cna, x_fprint, y_dep
        self.bs = batch_size
        
    def __getitem__(self, idx):
        start, end = idx * self.bs, (idx+1) * self.bs
        return (self.x_mut[start:end], self.x_exp[start:end], self.x_cna[start:end], self.x_fprint[start:end]), self.y_dep[start:end]
    
class generator2(generator):
    def __init__(self, x_mut, x_exp, x_fprint, y_dep, batch_size):
        self.x_mut, self.x_exp, self.x_fprint, self.y_dep = x_mut, x_exp, x_fprint, y_dep
        self.bs = batch_size
        
    def __getitem__(self, idx):
        start, end = idx * self.bs, (idx+1) * self.bs
        return (self.x_mut[start:end], self.x_exp[start:end], self.x_fprint[start:end]), self.y_dep[start:end]

class generator1(generator):
    def __init__(self, x_mut, x_fprint, y_dep, batch_size):
        self.x_mut, self.x_fprint, self.y_dep = x_mut, x_fprint, y_dep
        self.bs = batch_size
        
    def __getitem__(self, idx):
        start, end = idx * self.bs, (idx+1) * self.bs
        return (self.x_mut[start:end], self.x_fprint[start:end]), self.y_dep[start:end]
    
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def model_train_vis(history):
    coeff = history.history['coeff_determination']
    val_coeff = history.history['val_coeff_determination']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(coeff))
    
    plt.plot(epochs, coeff, 'bo', label='Training')
    plt.plot(epochs, val_coeff, 'b', label='Validation')
    plt.title('Training and validation Coeff determination(R-squared)')
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training')
    plt.plot(epochs, val_loss, 'b', label='Validation')
    plt.title('Training and validation loss(MSE)')
    plt.legend()

    plt.show()


```
# with open('prediction/data/ccl_complete_data_28CCL_1298DepOI_36344samples_demo.pickle', 'rb') as f:
# with open('data/ccl_complete_data_278CCL_1298DepOI_360844samples.pickle', 'rb') as f:
#     data_mut, data_exp, data_cna, data_meth, data_dep, data_fprint = pickle.load(f)
# This pickle file is for DEMO ONLY (containing 28 CCLs x 1298 DepOIs = 36344 samples)!
# First 1298 samples correspond to 1298 DepOIs of the first CCL, and so on.
# For the complete data used in the paper (278 CCLs x 1298 DepOIs = 360844 samples),
# please substitute by 'ccl_complete_data_278CCL_1298DepOI_360844samples.pickle',
# to which a link can be found in README.md
```

## **Data Load & Path**

In [4]:
TRAIN_PATH = '/home/wmbio/WORK/gitworking/Dependency_prediction/preprocessing/TRAIN/2022-07-27/'
TEMP_PATH = "prediction/train_preprocessing/"
SAVE_PATH = "prediction/custom_model/"

In [5]:
with open('prediction/data/ccl_complete_data_28CCL_1298DepOI_36344samples_demo.pickle', 'rb') as f:
    data_mut, data_exp, data_cna, data_meth, data_dep, data_fprint = pickle.load(f)

## **Build Model**

* **Pretrained Model load - TCGA**

In [6]:
premodel_mut = pickle.load(open('prediction/data/premodel_tcga_mut_1000_100_50.pickle', 'rb'))
premodel_exp = pickle.load(open('prediction/data/premodel_tcga_exp_500_200_50.pickle', 'rb'))
premodel_cna = pickle.load(open('prediction/data/premodel_tcga_cna_500_200_50.pickle', 'rb'))
premodel_meth = pickle.load(open('prediction/data/premodel_tcga_meth_500_200_50.pickle', 'rb'))
print("\n\nDatasets successfully loaded.")



Datasets successfully loaded.


In [7]:
activation_func = 'relu' # for all middle layers
activation_func2 = 'linear' # for output layer to output unbounded gene-effect scores
init = 'he_uniform'
dense_layer_dim = 250
batch_size = 500
num_epoch = 100
num_DepOI = 1298 # 1298 DepOIs as defined in our paper
num_ccl = int(data_mut.shape[0]/num_DepOI)

In [8]:
id_rand = np.random.permutation(num_ccl)
id_cell_train = id_rand[np.arange(0, round(num_ccl*0.9))]
id_cell_test = id_rand[np.arange(round(num_ccl*0.9), num_ccl)]

# prepare sample indices (selected CCLs x 1298 DepOIs)
id_train = np.arange(0, 1298) + id_cell_train[0]*1298
for y in id_cell_train:
    id_train = np.union1d(id_train, np.arange(0, 1298) + y*1298)
id_test = np.arange(0, 1298) + id_cell_test[0] * 1298
for y in id_cell_test:
    id_test = np.union1d(id_test, np.arange(0, 1298) + y*1298)
print("\n\nTraining/validation on %d samples (%d CCLs x %d DepOIs) and testing on %d samples (%d CCLs x %d DepOIs).\n\n" % (
    len(id_train), len(id_cell_train), num_DepOI, len(id_test), len(id_cell_test), num_DepOI))



Training/validation on 32450 samples (25 CCLs x 1298 DepOIs) and testing on 3894 samples (3 CCLs x 1298 DepOIs).




In [ ]:
# tf.keras.utils.plot_model(model_final, show_shapes=True,  show_dtype=True, to_file='model_custom.png')

## **Fit & Evaluation Model**

### **(1) Full Model 4-Omics**

In [ ]:
train_generator = generator(data_mut[id_train], data_exp[id_train], data_cna[id_train], data_meth[id_train], 
                                    data_fprint[id_train], data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator(data_mut[id_test], data_exp[id_test], data_cna[id_test], data_meth[id_test], 
                                    data_fprint[id_test], data_dep[id_test], batch_size = batch_size)

In [ ]:
model = exp_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
SAVE_NAME = "model_custom_full_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

In [ ]:
model_train_vis(history=history)

### **(2) Expression-Mutation-CNA Model**

In [ ]:
train_generator = generator3(data_mut[id_train], data_exp[id_train], data_cna[id_train], 
                                    data_fprint[id_train], data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator3(data_mut[id_test], data_exp[id_test], data_cna[id_test],
                                    data_fprint[id_test], data_dep[id_test], batch_size = batch_size)

In [ ]:
model = exp_mut_cna_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
SAVE_NAME = "model_custom_mut_exp_cna_0725"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

In [ ]:
model_train_vis(history=history)

### **(3) Expression-Mutation Model**

In [ ]:
train_generator = generator2(data_mut[id_train], data_exp[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator2(data_mut[id_test], data_exp[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = exp_mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_mut_exp_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(4) Expression Model**

In [9]:
train_generator = generator1(data_exp[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [10]:
test_generator = generator1(data_exp[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [11]:
with tf.device('/cpu:0'):
    model = exp_model()
    es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
    model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
    history = model.fit(train_generator, 
                        validation_data=test_generator, 
                        epochs=num_epoch, verbose=1,         
                        callbacks = [es])

2022-07-28 12:33:57.556212: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-28 12:33:57.556752: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-28 12:33:57.595439: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-28 12:33:57.595965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.807GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2022-07-28 12:33:57.595975: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-28 12:33:57.596975: I tensorflow/stream_executor/platform/d

Epoch 1/100
65/65 [==============================] - 2s 22ms/step - loss: 2.2906 - coeff_determination: -10.1151 - val_loss: 0.1290 - val_coeff_determination: 0.3059
Epoch 2/100
65/65 [==============================] - 1s 19ms/step - loss: 0.1224 - coeff_determination: 0.3554 - val_loss: 0.0518 - val_coeff_determination: 0.7148
Epoch 3/100
65/65 [==============================] - 1s 19ms/step - loss: 0.0493 - coeff_determination: 0.7328 - val_loss: 0.0475 - val_coeff_determination: 0.7365
Epoch 4/100
65/65 [==============================] - 1s 20ms/step - loss: 0.0447 - coeff_determination: 0.7579 - val_loss: 0.0467 - val_coeff_determination: 0.7418
Epoch 5/100
65/65 [==============================] - 1s 19ms/step - loss: 0.0415 - coeff_determination: 0.7729 - val_loss: 0.0478 - val_coeff_determination: 0.7380
Epoch 6/100
65/65 [==============================] - 1s 19ms/step - loss: 0.0442 - coeff_determination: 0.7667 - val_loss: 0.0477 - val_coeff_determination: 0.7371
Epoch 7/100
65

In [12]:
SAVE_NAME = "model_paper_exp_0727_keras2"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))



Full DeepDEP model saved in prediction/custom_model/model_paper_exp_0727_keras2.h5




* **Keras 1.x**

In [ ]:
model = exp_model()

In [ ]:
history = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')
model.compile(loss='mse', optimizer='adam')
model.fit([data_exp[id_train], data_fprint[id_train]], data_dep[id_train], nb_epoch=num_epoch,
                validation_split=1/9, batch_size=batch_size, shuffle=True, callbacks=[history])

In [ ]:
SAVE_NAME = "model_paper_exp_0727"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

In [ ]:
import h5py

### **(5) Mutation Model**

In [ ]:
train_generator = generator1(data_mut[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator1(data_mut[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_mut_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(6) CNA Model**

In [ ]:
train_generator = generator1(data_cna[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator1(data_cna[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_cna_0726"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))